<a href="https://colab.research.google.com/github/VOTUANANH01/DS104_Distributed_and_parallel_computing/blob/main/GraphFrame_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204.2 MB 31 kB/s 
     |████████████████████████████████| 198 kB 13.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=e45e936fd12a014fa11e4627bcab9344d3fee67af6017b859168bbc1240a4505
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
%cd /usr/local/lib/python3.7/dist-packages/pyspark/jars
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar


/usr/local/lib/python3.7/dist-packages/pyspark/jars
--2022-09-20 12:57:52--  https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 54.230.18.61, 54.230.18.122, 54.230.18.120, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|54.230.18.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248133 (242K) [binary/octet-stream]
Saving to: ‘graphframes-0.8.1-spark3.0-s_2.12.jar’

graphframes-0.8.1-s 100%[===================>] 242.32K  --.-KB/s    in 0.03s   

2022-09-20 12:57:52 (7.73 MB/s) - ‘graphframes-0.8.1-spark3.0-s_2.12.jar’ saved [248133/248133]



In [3]:
%cd '/content/'
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar

/content
--2022-09-20 12:57:53--  https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 54.230.18.61, 54.230.18.122, 54.230.18.120, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|54.230.18.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248133 (242K) [binary/octet-stream]
Saving to: ‘graphframes-0.8.1-spark3.0-s_2.12.jar’

graphframes-0.8.1-s 100%[===================>] 242.32K  --.-KB/s    in 0.03s   

2022-09-20 12:57:53 (7.44 MB/s) - ‘graphframes-0.8.1-spark3.0-s_2.12.jar’ saved [248133/248133]



In [4]:
!jar xf /content/graphframes-0.8.1-spark3.0-s_2.12.jar

In [5]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.getOrCreate()


In [6]:
vertices=spark.createDataFrame(
    [
          ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)
    ],["id","name","age"]
)

In [7]:
edges=spark.createDataFrame(
    [
        ("a", "b", "friend"),
     ("b","a","friend"),
  # ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d","e","friend"),
  ("d", "a", "friend"),
  ("a", "d", "friend"),
  ("a", "e", "friend"),
  ("e", "a", "friend")
    ],["src", "dst", "relationship"]
)

In [8]:
import os

path = os.getcwd()

print(path)

/content


In [9]:
from graphframes import *
g = GraphFrame(vertices,edges)
print()

In [10]:
g.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  a|      friend|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  e|      friend|
|  d|  a|      friend|
|  a|  d|      friend|
|  a|  e|      friend|
|  e|  a|      friend|
+---+---+------------+



In [11]:
g.edges.filter("relationship='follow'").show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
+---+---+------------+



In [12]:
#show pairs of users with friend relationship
#not filter the same friendship
g.edges.filter("relationship='friend'").show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  a|      friend|
|  e|  d|      friend|
|  d|  e|      friend|
|  d|  a|      friend|
|  a|  d|      friend|
|  a|  e|      friend|
|  e|  a|      friend|
+---+---+------------+



In [13]:
#Find the youngest user's age in the graph
g.vertices.groupBy().min("age").show()

+--------+
|min(age)|
+--------+
|      29|
+--------+



#Motif Finding

In [14]:
#search for pairs of vertices with edges in both direction between them
motif=g.find("(a)-[e]->(b);(b)-[e2]->(a)")
motif.show()

+---------------+--------------+---------------+--------------+
|              a|             e|              b|            e2|
+---------------+--------------+---------------+--------------+
| [d, David, 29]|[d, e, friend]|[e, Esther, 32]|[e, d, friend]|
| [a, Alice, 34]|[a, e, friend]|[e, Esther, 32]|[e, a, friend]|
| [a, Alice, 34]|[a, b, friend]|   [b, Bob, 36]|[b, a, friend]|
| [d, David, 29]|[d, a, friend]| [a, Alice, 34]|[a, d, friend]|
|[e, Esther, 32]|[e, d, friend]| [d, David, 29]|[d, e, friend]|
| [a, Alice, 34]|[a, d, friend]| [d, David, 29]|[d, a, friend]|
|   [b, Bob, 36]|[b, a, friend]| [a, Alice, 34]|[a, b, friend]|
|[e, Esther, 32]|[e, a, friend]| [a, Alice, 34]|[a, e, friend]|
+---------------+--------------+---------------+--------------+



In [15]:
#vertices and edges can be anonymous
g.find("(a)-[]->(b);(b)-[]->(a)").show()

+---------------+---------------+
|              a|              b|
+---------------+---------------+
| [d, David, 29]|[e, Esther, 32]|
| [a, Alice, 34]|[e, Esther, 32]|
| [a, Alice, 34]|   [b, Bob, 36]|
| [d, David, 29]| [a, Alice, 34]|
|[e, Esther, 32]| [d, David, 29]|
| [a, Alice, 34]| [d, David, 29]|
|   [b, Bob, 36]| [a, Alice, 34]|
|[e, Esther, 32]| [a, Alice, 34]|
+---------------+---------------+



In [16]:
#more complex queries can be expressed by applying filter
motif.filter("b.age > 30").show()

+---------------+--------------+---------------+--------------+
|              a|             e|              b|            e2|
+---------------+--------------+---------------+--------------+
| [d, David, 29]|[d, e, friend]|[e, Esther, 32]|[e, d, friend]|
| [a, Alice, 34]|[a, e, friend]|[e, Esther, 32]|[e, a, friend]|
| [a, Alice, 34]|[a, b, friend]|   [b, Bob, 36]|[b, a, friend]|
| [d, David, 29]|[d, a, friend]| [a, Alice, 34]|[a, d, friend]|
|   [b, Bob, 36]|[b, a, friend]| [a, Alice, 34]|[a, b, friend]|
|[e, Esther, 32]|[e, a, friend]| [a, Alice, 34]|[a, e, friend]|
+---------------+--------------+---------------+--------------+



In [17]:
#each vertices and edge
motif.selectExpr("a.id as a","b.id as b").show()

+---+---+
|  a|  b|
+---+---+
|  d|  e|
|  a|  e|
|  a|  b|
|  d|  a|
|  e|  d|
|  a|  d|
|  b|  a|
|  e|  a|
+---+---+



In [18]:
#one way relationship
g.find("(a)-[e]->(b);!(b)-[]->(a)").selectExpr("a.id ","e.relationship",\
                                                "b.id "
).show()

+---+------------+---+
| id|relationship| id|
+---+------------+---+
|  e|      follow|  f|
|  c|      follow|  b|
|  f|      follow|  c|
+---+------------+---+



In [19]:
from functools import reduce
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

chain4=g.find("(a)-[ab]->(b);(b)-[bc]->(c);(c)-[cd]->(d)")


In [20]:
from IPython.core.magics.code import chain
#query on sequences, with state (cnt)
# (a) define method for updating state given the next element of the motif
sumFriends=lambda cnt,relationship: when(relationship=="friend",cnt+1).otherwise(cnt)

# (b) Use sequence operation to apply method to sequence of elements in motif
condition=reduce(lambda cnt,e: sumFriends(cnt,col(e).relationship),["ab","bc","cd"],lit(0))
                 
#(c) Apply filter to DataFrame
chainWith2Friends2=chain4.where(condition >=2)
chainWith2Friends2.show()

+----------------+--------------+---------------+--------------+---------------+--------------+---------------+
|               a|            ab|              b|            bc|              c|            cd|              d|
+----------------+--------------+---------------+--------------+---------------+--------------+---------------+
| [e, Esther, 32]|[e, d, friend]| [d, David, 29]|[d, e, friend]|[e, Esther, 32]|[e, f, follow]| [f, Fanny, 36]|
|  [a, Alice, 34]|[a, d, friend]| [d, David, 29]|[d, e, friend]|[e, Esther, 32]|[e, f, follow]| [f, Fanny, 36]|
| [e, Esther, 32]|[e, a, friend]| [a, Alice, 34]|[a, e, friend]|[e, Esther, 32]|[e, f, follow]| [f, Fanny, 36]|
|  [d, David, 29]|[d, a, friend]| [a, Alice, 34]|[a, e, friend]|[e, Esther, 32]|[e, f, follow]| [f, Fanny, 36]|
|    [b, Bob, 36]|[b, a, friend]| [a, Alice, 34]|[a, e, friend]|[e, Esther, 32]|[e, f, follow]| [f, Fanny, 36]|
|  [d, David, 29]|[d, e, friend]|[e, Esther, 32]|[e, d, friend]| [d, David, 29]|[d, e, friend]|[e, Esthe

#SubGraph

In [21]:
#select subgrah of user older than 30, and relationship of type " friend"
#drop isolated vertices (user) which are not contained in any edges (relationship)

g1=g.filterVertices("age>30").\
  filterEdges("relationship ='friend'").\
  dropIsolatedVertices()


In [22]:
g1.vertices.show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  e|Esther| 32|
|  b|   Bob| 36|
|  a| Alice| 34|
+---+------+---+



In [23]:
#select subgraph based on edges "e" of type "follow"
# pointing from a younger user "a" to an older user "b"

paths=g.find("(a)-[e]->(b)")\
  .filter("e.relationship ='follow'")\
  .filter("a.age < b.age")
#"paths" contains vertex info. Extract the edges
e2=paths.select("e.src","e.dst","e.relationship")

#construct the subgrah
g2=GraphFrame(g.vertices,e2)

In [24]:
g2.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+



In [25]:
g2.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  e|  f|      follow|
|  c|  b|      follow|
+---+---+------------+



#Breath-first search (BFS)

In [26]:
#search from "Esther" for user of age < 32
paths=g.bfs("name='Esther'","age <32")
paths.show()

#specify edge filters or max path lengths
path2=g.bfs("name='Esther'","age <32",edgeFilter="relationship!='friend'"\
  ,maxPathLength=3)
path2.show()

+---------------+--------------+--------------+
|           from|            e0|            to|
+---------------+--------------+--------------+
|[e, Esther, 32]|[e, d, friend]|[d, David, 29]|
+---------------+--------------+--------------+

+---------------+--------------+--------------+--------------+----------------+
|           from|            e0|            v1|            e1|              to|
+---------------+--------------+--------------+--------------+----------------+
|[e, Esther, 32]|[e, f, follow]|[f, Fanny, 36]|[f, c, follow]|[c, Charlie, 30]|
+---------------+--------------+--------------+--------------+----------------+



#Connect components

In [31]:
!pwd

/content


In [33]:
!mkdir /content/checkpoints


In [34]:
spark.sparkContext.setCheckpointDir('/content/checkpoints')


In [35]:

result=g.connectedComponents()
result.select("id","component").orderBy("component").show()

+---+------------+
| id|   component|
+---+------------+
|  g|146028888064|
|  a|412316860416|
|  b|412316860416|
|  c|412316860416|
|  d|412316860416|
|  f|412316860416|
|  e|412316860416|
+---+------------+



#strongly connected components

In [39]:
result=g.stronglyConnectedComponents(maxIter=10)
result.select("id","component").orderBy("component").show()

+---+------------+
| id|   component|
+---+------------+
|  g|146028888064|
|  b|412316860416|
|  a|412316860416|
|  e|412316860416|
|  f|412316860416|
|  d|412316860416|
|  c|412316860416|
+---+------------+



#Label Propagation ALgorithm (LPA)

In [40]:
result = g.labelPropagation(maxIter=5)
result.select("id","label").show()

+---+-------------+
| id|        label|
+---+-------------+
|  g| 146028888064|
|  b|1460288880640|
|  e|1460288880640|
|  a|1460288880640|
|  f|1460288880640|
|  d|1460288880640|
|  c|1460288880640|
+---+-------------+



#PageRank

In [44]:
#Run PageRank until convergence to tolerance "tol"
results=g.pageRank(resetProbability=0.15,tol=0.01)

#Display resulting pageranks and final edge weights
results.vertices.select("id","pagerank").show()
results.edges.select("src","dst","weight").show()

#Run pagerank for a fied numberof iterations

results2=g.pageRank(resetProbability=0.15,maxIter=10)

#Run PageRank personalized for vertex "a"
result3=g.pageRank(resetProbability=0.15,maxIter=10,sourceId="a")

#Run PageRank personalized for vertex ("a","b","c","d") in parallel
result4=g.parallelPersonalizedPageRank(resetProbability=0.15,sourceIds=["a","b","c","d"],maxIter=10)

+---+-------------------+
| id|           pagerank|
+---+-------------------+
|  g|0.18618661660049637|
|  b| 1.2658584908762605|
|  e| 1.2379542381046424|
|  a| 2.0618615072239352|
|  f| 0.5341868752251782|
|  d|  1.115401191774445|
|  c| 0.5985510801950417|
+---+-------------------+

+---+---+------------------+
|src|dst|            weight|
+---+---+------------------+
|  b|  a|               1.0|
|  a|  b|0.3333333333333333|
|  d|  e|               0.5|
|  e|  a|0.3333333333333333|
|  e|  f|0.3333333333333333|
|  e|  d|0.3333333333333333|
|  a|  d|0.3333333333333333|
|  c|  b|               1.0|
|  a|  e|0.3333333333333333|
|  f|  c|               1.0|
|  d|  a|               0.5|
+---+---+------------------+



#Shortest path

In [45]:
spark.sql("set spark.sql.legacy.allowUntyScalaUDF=true")

DataFrame[key: string, value: string]

In [47]:
results=g.shortestPaths(landmarks=["a","d"])
results.select("id","distances").show()

+---+----------------+
| id|       distances|
+---+----------------+
|  g|              []|
|  b|[a -> 1, d -> 2]|
|  e|[a -> 1, d -> 1]|
|  a|[a -> 0, d -> 1]|
|  f|[a -> 3, d -> 4]|
|  d|[a -> 1, d -> 0]|
|  c|[a -> 2, d -> 3]|
+---+----------------+



#Triangle count

In [48]:
results= g.triangleCount()
results.select("id","count").show()

+---+-----+
| id|count|
+---+-----+
|  g|    0|
|  f|    0|
|  e|    1|
|  d|    1|
|  c|    0|
|  b|    0|
|  a|    1|
+---+-----+



#Saving and loading GraphFrames

In [49]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark.sparkContext)
#save vertices and edges as Parquet to some location
g.vertices.write.parquet("quickstart/vertices")
g.edges.write.parquet("quickstart/edges")

#load the vertices  and edges back
sameV=sqlContext.read.parquet("quickstart/vertices")
sameE=sqlContext.read.parquet("quickstart/edges")

#create an indentical GraphFrame
sameG=GraphFrame(sameV,sameE)

In [50]:
sameG.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
+---+-------+---+



#Message passing via AggregateMessages



In [57]:
from pyspark.sql.functions import sum as sqlsum
from graphframes.lib import AggregateMessages as AM


In [58]:
msgToSrc=AM.dst["age"]
msgToDst=AM.src["age"]
agg = g.aggregateMessages(
    sqlsum(AM.msg).alias("summedAges"),
    sendToSrc=msgToSrc,
    sendToDst=msgToDst)
agg.show()

+---+----------+
| id|summedAges|
+---+----------+
|  f|        62|
|  e|       162|
|  d|       132|
|  c|        72|
|  b|        98|
|  a|       194|
+---+----------+

